In [20]:
import numpy as np

import spglib as spg

from hilde.io import read
from hilde.harmonic_analysis import HarmonicAnalysis
from hilde.structure.convert import to_spglib_cell
from hilde.helpers.numerics import clean_matrix
from hilde.helpers.lattice_points import get_commensurate_q_points

from hilde.spglib.q_mesh import get_ir_reciprocal_mesh

In [21]:
mat = 'mgo'
primitive = read(f'{mat}/geometry.in')
supercell = read(f'{mat}/geometry.in.supercell')

In [22]:
ha = HarmonicAnalysis(primitive, supercell)

Set up harmonic analysis for MgO:
.. found 32 (31) lattice points in 0.002s
.. found 32 (31) lattice points in 0.001s
** Force constants not set, your choice.
.. time elapsed: 0.141s


In [23]:
my_mesh = np.round(ha.q_points @ supercell.cell.T).astype(int)
n_lp = len(my_mesh)
my_mesh, n_lp

(array([[ 0,  0,  0],
        [ 1,  0,  0],
        [ 0,  1,  0],
        [ 0,  0,  1],
        [ 1,  1,  0],
        [ 1,  0,  1],
        [ 0,  1,  1],
        [ 1,  1, -1],
        [ 1, -1,  1],
        [-1,  1,  1],
        [ 1,  1,  1],
        [ 2,  0,  0],
        [ 0,  2,  0],
        [ 0,  0,  2],
        [ 2,  1,  0],
        [ 1,  2,  0],
        [ 2,  0,  1],
        [ 0,  2,  1],
        [ 1,  0,  2],
        [ 0,  1,  2],
        [ 2,  1, -1],
        [ 1,  2, -1],
        [ 2, -1,  1],
        [ 1, -1,  2],
        [ 2,  1,  1],
        [-1,  2,  1],
        [ 1,  2,  1],
        [-1,  1,  2],
        [ 1,  1,  2],
        [ 3,  0,  0],
        [ 0,  3,  0],
        [ 0,  0,  3]]), 32)

In [24]:
n_qmesh = my_mesh.max(axis=0) - my_mesh.min(axis=0) + 1
n_qmesh

array([5, 5, 5])

In [25]:
mapping, spg_mesh = spg.get_ir_reciprocal_mesh(n_qmesh, to_spglib_cell(primitive))

In [26]:
# create the list of irreducible grid points and the mapping to the reduced list

# irreducible grid points
ir_grid = spg_mesh[np.unique(mapping)]

# dict translates between full and reduced grid point indices
dct = {}
for nn, ii in enumerate(np.unique(mapping)):
    dct[ii] = nn

# mapping to the reduced indices
ir_mapping = np.array([dct[ii] for ii in mapping])

In [27]:
# verify that this mapping works
all((spg_mesh[q] == ir_grid[ir_mapping[i]]).all() for i, q in enumerate(mapping))

True

In [28]:
# match the q points from my list with the ones from spglib

match_list = -np.ones(n_lp, dtype=int)
my_mapping = -np.ones(n_lp, dtype=int)

for i1, q1 in enumerate(my_mesh % n_qmesh):
    for i2, q2 in enumerate(spg_mesh % n_qmesh):
        if np.linalg.norm(q1 - q2) < 1e-12:
            # exchange the q point in spg list with my one
            ir_grid[ir_mapping[i2]] = q1
            match_list[i1] = i2
            my_mapping[i1] = ir_mapping[i2]

# sanity checks
assert -1 not in match_list, match_list
assert -1 not in my_mapping, my_mapping
assert len(np.unique(match_list)) == n_lp, (len(np.unique(match_list)), match_list)

In [29]:
match_list

array([  0,   1,   5,  25,   6,  26,  30, 106,  46,  34,  31,   2,  10,
        50,   7,  11,  27,  35,  51,  55, 107, 111,  47,  71,  32,  39,
        36,  59,  56,   3,  15,  75])

In [30]:
ir_grid[np.unique(my_mapping)]

array([[0, 0, 0],
       [1, 1, 1],
       [0, 0, 3],
       [0, 1, 1],
       [0, 1, 2],
       [1, 1, 2],
       [4, 1, 2]], dtype=int32)

In [31]:
# dict to bring order into the chaos
dct = {}
for nn, ii in enumerate(np.unique(my_mapping)):
    dct[ii] = nn

In [32]:
dct

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 6: 5, 9: 6}

In [33]:
my_ir_grid = ir_grid[np.unique(my_mapping)]
len(my_ir_grid), len(np.unique(my_ir_grid, axis=0))

(7, 7)

In [34]:
my_ir_mapping = np.array([dct[ii] for ii in my_mapping])
my_ir_mapping

array([0, 1, 1, 1, 3, 3, 3, 4, 4, 4, 1, 2, 2, 2, 4, 4, 4, 4, 4, 4, 6, 6,
       6, 6, 5, 6, 5, 6, 5, 2, 2, 2])

In [35]:
# verify
all((my_ir_grid[my_ir_mapping[i]] % n_qmesh == ir_grid[q] % n_qmesh).all() for i, q in enumerate(my_mapping))

True

In [36]:
print(f'Number of q points reduced from\n  {len(my_ir_mapping)} to {len(np.unique(my_ir_mapping))}')

Number of q points reduced from
  32 to 7


In [37]:
# now implemented in HarmonicAnalysis
m, iqs = ha.irreducible_q_points

.. number of q points reduced from 32 to 7 in 0.050s


In [38]:
for fq in iqs:
    cart_q = fq @ supercell.get_reciprocal_cell()
    frac_q = clean_matrix(cart_q @ primitive.cell.T)
    # map back to supercell
    frac_q_supercell = clean_matrix(((frac_q + .01) % 1 - .01) @ primitive.get_reciprocal_cell() @ supercell.cell.T)
    print(fq, frac_q, frac_q_supercell)

[0 0 0] [0. 0. 0.] [0. 0. 0.]
[1 1 1] [0.5 0.5 0.5] [1. 1. 1.]
[0 0 3] [0.75 0.75 0.  ] [0. 0. 3.]
[0 1 1] [0.5  0.25 0.25] [0. 1. 1.]
[0 1 2] [0.75 0.5  0.25] [0. 1. 2.]
[1 1 2] [0.75 0.75 0.5 ] [1. 1. 2.]
[-1  1  2] [0.75 0.25 0.  ] [-1.  1.  2.]
